# Functions

In [ ]:
from datetime import time
import datetime as dt

def adddatecolumns(data,pf) :
    data['year'] = pf['CSTARTTIME'].apply(lambda x: x.date().year)
    data['month'] = pf['CSTARTTIME'].apply(lambda x: x.date().month)
    data['day'] = pf['CSTARTTIME'].apply(lambda x: x.date().day)
    data['hour'] = pf['CSTARTTIME'].apply(lambda x: x.time().hour)
    data['minute'] = pf['CSTARTTIME'].apply(lambda x: x.time().minute)
    data['second'] = pf['CSTARTTIME'].apply(lambda x: x.time().second)
    data['microsecond'] = pf['CSTARTTIME'].apply(lambda x: x.time().microsecond)

def converttimestampcolumnn(pf,tsc) :
    pf[tsc] = pf[tsc].apply(lambda x: dt.datetime.fromtimestamp(float(x) / 1e3))

In [ ]:
## Setup charts
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

#vis functions

def label(graph,skip,rot) :
    for ind, label in enumerate(graph.get_xticklabels()):
        if ind % skip == 0:  # every 10th label is kept
            label.set_visible(True)
            label.set_rotation(rot)
        else:
            label.set_visible(False)
            
def abc(mdcountsall) :
    a = mdcountsall.index.get_level_values(0).astype(str)
    b = mdcountsall.index.get_level_values(1).astype(str)
    c = mdcountsall.index.get_level_values(2).astype(str)
    return a + "-" + b + "-" + c 

def abcd(mdcountsall) :
    return abc(mdcountsall) + "-" + mdcountsall.index.get_level_values(3).astype(str)

def ymdh(mdcountsall) :
    return abcd(mdcountsall)

def get_ymdh(mdcountsall) :
    a = mdcountsall.index.get_level_values(0).astype(str)
    b = mdcountsall.index.get_level_values(1).astype(str)
    c = mdcountsall.index.get_level_values(2).astype(str)
    d = mdcountsall.index.get_level_values(3).astype(str)
    return a,b,c,d

def get_ymdh_string(a,b,c,d) :
    return a + "-" + b + "-" + c + "-" + d

def get_ym(mdcountsall) :
    a = mdcountsall.index.get_level_values(0).astype(str)
    b = mdcountsall.index.get_level_values(1).astype(str)
    return a,b

def get_ym_string(a,b) :
    return a + "-" + b

In [ ]:

#mdcountsall=pfall[(pfall['month'] == 2) & (pfall['day'] > 16)].groupby(['year','month','day','hour'])['outcome'].count()
#mdcountsall=pfall[(pfall['month'] == 2)].groupby(['year','month','day','hour'])['outcome'].count()
#mdcountsall=pfall.groupby(['year','month','day'])['outcome'].count()

def createData(pfall,month,outcome) :
    if outcome < 2 :
        mdcountsall = pfall[(pfall['month'] == month) & (pfall['outcome'] == outcome)].groupby(['year','month','day','hour'])['outcome'].count()
    else :
        if (month > 0) & (month < 13) :
            mdcountsall = pfall[(pfall['month'] == month)].groupby(['year','month','day','hour'])['outcome'].count()
        else :
            mdcountsall = pfall.groupby(['year','month','day','hour'])['outcome'].count()    
    a,b,c,d = get_ymdh(mdcountsall)
    data2 = pd.DataFrame()
    data2['date'] = get_ymdh_string(a,b,c,d)
    data2['outcome'] =  mdcountsall.reset_index()['outcome'].astype(int) 

    #for pivot table
    data2['hours'] =  d.astype(int) 
    data2['days']  =  c.astype(int) 
    return data2

def createData_ym(pfall,month,outcome) :
    if outcome < 2 :
        mdcountsall = pfall[(pfall['outcome'] == outcome)].groupby(['year','month'])['outcome'].count()
    else :
        if (month > 0) & (month < 13) :
            mdcountsall = pfall[(pfall['month'] == month)].groupby(['year','month'])['outcome'].count()
        else :
            mdcountsall = pfall.groupby(['year','month'])['outcome'].count()    
    a,b = get_ym(mdcountsall)
    data2 = pd.DataFrame()
    data2['date'] = get_ym_string(a,b)
    data2['outcome'] =  mdcountsall.reset_index()['outcome'].astype(int) 

    #for pivot table
    #data2['hours'] =  d.astype(int) 
    #data2['days']  =  c.astype(int) 
    return data2

## heatmap
def createHeatmap(piv,title="") :
    plt.figure(figsize=(24,8))
    plt.title(title)
    ax = sns.heatmap(piv, square=True)
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=0 )
    plt.tight_layout()
    plt.show()
    return ax

def createBarplot(md,fx,fy,fontscale,title="") :
    sns.set(style='whitegrid', palette='muted', font_scale=fontscale)
    plt.figure(figsize=(fx,fy))
    plt.title(title)
    ax = sns.barplot(x=md['date'], y=md['outcome'], data=md)
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=0 )
    plt.tight_layout()
    plt.show()
    return ax

# kernel density estimate (KDE) 
def createKDE(data2,fx,fy,fontscale,title="") :
    sns.set(style='whitegrid', palette='muted', font_scale=fontscale)
    plt.figure(figsize=(fx,fy))
    plt.title(title)
    # Histogram 
    #ax = sns.distplot(a=data2['outcome'], kde=False)
    ax = sns.kdeplot(data=data2['outcome'], shade=True)
    plt.tight_layout()
    plt.show()
    return ax

Time Series Anomaly Detection with LSTM Autoencoders

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
# functions

OUTCOME = 'close'

TIME_STEPS = 24
#TIME_STEPS = 30
#TIME_STEPS = 720
#TIME_STEPS = 168
#TIME_STEPS = 336

# setup data (current)
def createDataframe(pfall) :
    data3 = createData(pfall,0,2)
    df = pd.DataFrame()
    df[OUTCOME] = data3['outcome']
    df.set_index(data3['date'], inplace=True)
    return df

def getTrainAndTest(df,TRAIN_SIZE) :
    train_size = int(len(df) * TRAIN_SIZE)
    test_size = len(df) - train_size
    train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
    print("train.shape: ",train.shape, "test.shape: ", test.shape)
    return train, test

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

def initmodel():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(
        units=64, 
        input_shape=(X_train.shape[1], X_train.shape[2])
    ))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.RepeatVector(n=X_train.shape[1]))
    model.add(keras.layers.LSTM(units=64, return_sequences=True))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=X_train.shape[2])))
    model.compile(loss='mae', optimizer='adam')
    return model

def testScoreDF(model, THRESHOLD) : 
    X_test_pred = model.predict(X_test)
    test_mae_loss = np.mean(np.abs(X_test_pred - X_test), axis=1)

    test_score_df = pd.DataFrame(index=test[TIME_STEPS:].index)
    test_score_df['loss'] = test_mae_loss
    test_score_df['threshold'] = THRESHOLD
    test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
    test_score_df[OUTCOME] = test[TIME_STEPS:][OUTCOME]
    return test_score_df

# Start

In [ ]:
import numpy as np
import pandas as pd

import findspark
findspark.init()

import pyspark
import random

from pyspark import SparkContext
from pyspark.sql import SQLContext

#sc = pyspark.SparkContext(appName="Pi")

#sc = SparkContext()
#sqlContext = SQLContext(sc)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

sparkSession = SparkSession.builder.config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.driver.maxResultSize", "0").appName("example-pyspark-read-and-write").getOrCreate()


df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*')

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

cat_features = ['CSERVICE','CRECEIVERPROTOCOL','CSENDERPROTOCOL','CSENDERENDPOINTID','CRECEIVERENDPOINTID','CSTATUS']

def encodeall(pfall,cat_features):
    #Prepping categorical variables
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    # Apply the label encoder to each column
    encodedpfall = pfall[cat_features].apply(encoder.fit_transform)
    return encodedpfall
  
def getpfbasic(pfall) :   
    #pfall = df2.limit(5000000).toPandas()    
    #print(pfall.loc[pfall['CSTARTTIME'].idxmax()]['CSTARTTIME'],len(pfall.index))
    pfall = pfall.assign(outcome=(~( ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'InvoicePortal')) | ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'IDS')) | (pfall['CSTATUS'] == 'SUCCESS') | (pfall['CSTATUS'] == 'SUCCESS_DOWNLOADED') | (pfall['CSTATUS'] == 'SUCCESS_POLLQUEUE'))).astype(int))
    converttimestampcolumnn(pfall,'CSTARTTIME')
    pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
    #pfall['CSENDERENDPOINTID'] = pfall['CSENDERENDPOINTID'].astype(str)   
    #pfall['CRECEIVERENDPOINTID'] = pfall['CRECEIVERENDPOINTID'].astype(str)
    pfall['CMESSAGETAT2'] = pfall['CMESSAGETAT2'].astype(int)
    pfall['CSLATAT'] = pfall['CSLATAT'].astype(int)
    pfall['CINBOUNDSIZE'] = pfall['CINBOUNDSIZE'].astype(int)
    return pfall

def getpf(df2) :    
    pfall = getpfbasic(df2)
    encodedpfall = encodeall(pfall,cat_features)
    dataall = pfall[['CGLOBALMESSAGEID','CMESSAGETAT2','CSLATAT','CINBOUNDSIZE', 'outcome']].join(encodedpfall)
    adddatecolumns(dataall,pfall)    
    return dataall

In [ ]:
import numpy as np
import hashlib
from pyspark.sql.functions import when, lit, col

selected = ['CSERVICE','CRECEIVERPROTOCOL','CSENDERPROTOCOL','CSENDERENDPOINTID','CRECEIVERENDPOINTID','CGLOBALMESSAGEID','CSTARTTIME','CMESSAGETAT2','CSLATAT','CINBOUNDSIZE','CSTATUS']

def getpfall(df,selected) :
    #pfall = df.limit(5000000).toPandas()  
    pfall = df.toPandas() 
    for each in selected:
        pfall[each] = pfall[each].astype(str)
    if len(pfall) == 0:
        return pfall,0,0
    return pfall, int(pfall.loc[pfall['CSTARTTIME'].astype(int).idxmax()]['CSTARTTIME']),len(pfall.index)        

def gettest(to,selected) :
    df2 = df.withColumn('CSTARTTIME', col('CSTARTTIME').cast('long')).filter(col("CSTARTTIME") < to ).select(selected).dropDuplicates().orderBy('CSTARTTIME')    
    return getpfall(df2,selected)

def getdata_lt(to,selected) :
    df2 = df.withColumn('CSTARTTIME', col('CSTARTTIME').cast('long')).filter(col("CSTARTTIME") < to ).select(selected).dropDuplicates().orderBy('CSTARTTIME') 
    #dataall = getpf(df2)    
    #return dataall
    return getpfall(df2,selected)
    
def getdata_gt(_from,_diff,selected) :
    to = _from + _diff
    df2 = df.withColumn('CSTARTTIME', col('CSTARTTIME').cast('long')).filter(col("CSTARTTIME") > _from ).filter(col("CSTARTTIME") < to ).select(selected).dropDuplicates().orderBy('CSTARTTIME') 
    #dataall = getpf(df2)
    #return dataall
    return getpfall(df2,selected)

start_lt = 1577041508174
start_gt = [1577041507723,1580032925655,1581929401680,1583136852562]
timestamp_diff = 1000000000

dataall2 = getdata_lt(start_lt,selected)
dataall = dataall2[0]

while True:
    dataall2 = getdata_gt(dataall2[1] ,timestamp_diff,selected)
    if dataall2[2] == 0:
        break  
    dataall = dataall.append(dataall2[0], ignore_index=True)

In [ ]:
pfall = getpf(dataall)

# Number messages sent by endpoints

In [ ]:
md = createData_ym(pfall,0,2)
createBarplot(md,24,9,3.0,title="number messages sent by all endpoints")

In [ ]:
md = createData_ym(pfall,0,1)
createBarplot(md,24,9,3.0,title="number messages with errors sent by all endpoints")

# Details for different CSENDERENDPOINTID

In [ ]:
#pd.unique(mdcountsall[1].index.get_level_values(0))
TOP=500000
result = pfall.groupby(['CSENDERENDPOINTID']).count()
data2 = pd.DataFrame()
data2['date'] = result.index.get_level_values(0).astype(str)
data2['outcome'] =  result['outcome'].astype(int)
topsender =  data2[data2['outcome'] > TOP].sort_values('outcome').reset_index()
topsender.columns = ['index', 'CSENDERENDPOINTID', 'outcome']

In [ ]:
#pfall1 = pfall[pfall['CSENDERENDPOINTID'].isin(topsender['date'])]
pfall1 = pfall[pfall['CSENDERENDPOINTID']==int(topsender.iloc[7]['CSENDERENDPOINTID'])]

In [ ]:
title = str(len(topsender)) + " senders with more than " + str(TOP) + " messages (kernel density estimate )"
ax = createKDE(data2,16,8,2,title)
label(ax,1,90)
#topsender.iloc[7]

# Number messages of selected endpoint (Msgs / h)

In [ ]:
print("CSENDERENDPOINTID: " + str(topsender.iloc[7]['CSENDERENDPOINTID']) + ": number messages so far = " + str(topsender.iloc[7]['outcome']))
        

def createHeatmapSeries(pfall, months,category, outcome) :
    for month in months:
        data2 = createData(pfall,month,outcome)
        piv = pd.pivot_table(data2, values="outcome",index=["hours"], columns=["days"], fill_value=0)
        #titlestring = "CSENDERENDPOINTID: " + str(topsender.iloc[7]['CSENDERENDPOINTID']) + ": "+ category + " so far = " + str(topsender.iloc[7]['outcome']) + " , month: " + str(month) 
        titlestring = "CSENDERENDPOINTID: " + str(topsender.iloc[7]['CSENDERENDPOINTID']) + ": "+ category  + " month: " + str(month) 
        sns.set(style='whitegrid', palette='muted', font_scale=1.2)
        createHeatmap(piv, titlestring)
       
createHeatmapSeries(pfall1,[10,11,12,1,2,3,4],'messages', 2)

# Number messages with errors of selected endpoint

In [ ]:
createHeatmapSeries(pfall1, [11,12,1,2,3,4],'messages with errors', 1)

# Anomaly Detection with LSTM Autoencoders (selected SENDERPOINTID)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
df = createDataframe(pfall1)
train, test = getTrainAndTest(df,0.95)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(train[[OUTCOME]])
train[OUTCOME] = scaler.transform(train[[OUTCOME]])
test[OUTCOME] = scaler.transform(test[[OUTCOME]])

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train[[OUTCOME]], train.close, TIME_STEPS)
X_test, y_test = create_dataset(test[[OUTCOME]], test.close, TIME_STEPS)
print(X_train.shape)

model = initmodel()

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    shuffle=False)

X_train_pred = model.predict(X_train)
train_mae_loss = np.mean(np.abs(X_train_pred - X_train), axis=1)

In [ ]:
sns.distplot(train_mae_loss, bins=50, kde=True);

In [ ]:
test_score_df = testScoreDF(model, 1.5)
anomalies     = test_score_df[test_score_df.anomaly == True]

In [ ]:
plt.plot(test_score_df.index, test_score_df.loss, label='loss')
plt.plot(test_score_df.index, test_score_df.threshold, label='threshold')
plt.xticks(rotation=25)
plt.legend();

In [ ]:
#anomalies.head()
#anomalies

In [ ]:
plt.plot(
  test[TIME_STEPS:].index, 
  scaler.inverse_transform(test[TIME_STEPS:].close), 
  label='msg count'
);

ax = sns.scatterplot(
  anomalies.index,
  scaler.inverse_transform(anomalies.close),
  color=sns.color_palette()[3],
  s=152,
  label='anomaly'
)
plt.xticks(rotation=25)
plt.legend();

label(ax,5,80)